# Submission Notebook

## Importing and Loading Evaluation Dataset

In [1]:
import certifi
import os
os.environ['SSL-CERT_FILE'] = certifi.where()
from giza_datasets import DatasetsLoader

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
from sklearn.linear_model import LinearRegression 

pd.set_option('display.max_columns', 200)

In [2]:
loader = DatasetsLoader()
df = loader.load("gora-competition-evaluation").to_pandas()


Dataset read from cache.
Loading dataset gora-competition-evaluation from cache.


In [4]:
features = ['total_borrow', 'count_borrow', 'avg_borrow_amount',
       'std_borrow_amount', 
       'borrow_amount_cv', 'total_repay', 'count_repay', 'avg_repay_amount',
       'std_repay_amount', 'repay_amount_cv', 'total_deposit', 'count_deposit',
       'avg_deposit_amount', 'std_deposit_amount', 'deposit_amount_cv',
       'total_redeem', 'count_redeem', 'avg_redeem_amount',
       'std_redeem_amount', 'redeem_amount_cv',
       'days_since_first_borrow', 'net_outstanding',
       'int_paid', 'net_deposits', 'count_repays_to_count_borrows',
       'avg_repay_to_avg_borrow', 'net_outstanding_to_total_borrowed',
       'net_outstanding_to_total_repaid', 'count_redeems_to_count_deposits',
       'total_redeemed_to_total_deposits', 'avg_redeem_to_avg_deposit',
       'net_deposits_to_total_deposits', 'net_deposits_to_total_redeemed',
       'dex_total_sum_added',
       'dex_total_sum_removed', 'dex_total_sum_swapped']

In [5]:
import joblib 
scaler = joblib.load('scaler.pkl')

## Predicting On Evaluation set

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

X_test = df[features].values
X_test_scaled = scaler.transform(X_test)
class CustomDataset(TensorDataset):
    def __init__(self, X):
        self.X = torch.tensor(X, dtype=torch.float)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

batch_size = 256

test_dataset = CustomDataset(X_test_scaled)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1) 

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = torch.load('model.pth', map_location=device)
model.to(device)

model.eval()
predictions = []
with torch.no_grad():
    for inputs in test_data_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())


## Saving Predictions

In [10]:
import pandas as pd 
predictions = np.array(predictions).flatten().clip(0, np.inf)
results = pd.Series(predictions)
df["total_liquidation_to_total_borrow"] = results

results.to_csv("submission.csv")